# Summarize Google News Results with LangChain🦜🔗, Huggingface🤗 and Serper API

## Overview

Text summarization is the process of creating a shorter version of a text document while still preserving the most important information. This can be useful for a variety of purposes, such as quickly skimming a long document, getting the gist of an article, or sharing a summary with others. LLMs can be used to create summaries of news articles, research papers, technical documents, and other types of text.

<img src="images/miztiik_text_summarization_01.png" width="50%"/>


## Chunking Strategies for LLM Applications

- **Stuffing method** - The `stuffing` method is the easiest way to summarize text by feeding the entire document to a large language model (LLM) in a single call. This method has both pros and cons.

  - **Pros**:
    - Only required a single call to the model, which can be faster than other methods that require multiple calls
    - When summarizing text, the model has access to all the data at once, which can result in a better summary.
  - **Cons**:
    - Most models have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.
    - This method only works on smaller pieces of data and not suitable to large documents most of the time.

- **MapReduce method** - It is a technique for summarizing large pieces of text by first summarizing smaller chunks of text and then combining those summaries into a single summary. The `MapReduce` method implements a multi-stage summarization. In LangChain, you can use `MapReduceDocumentsChain` as part of the `load_summarize_chain` method. What you need to do is setting `map_reduce` as `chain_type` of your chain.
  - MapReduce with Overlapping Chunks method
  - MapReduce with Rolling Summary method
  
![Miztiik Automation: Text Summarization](images/miztiik_automation_docs_copilot_using_llm_rag_02.png)

As we do not know the length of the document, we will use the `map-reduce` method to summarize the news articles.

In this notebook, we will try fetch the latest Google news using server API and use AI-generated summaries with LangChain LLM framework or huggingface transformers.


<a href="https://colab.research.google.com/github/miztiik/llm-bootcamp/blob/main/chapters/text_summarization/news_summarization_with_hf.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
%%capture
# Comment the above line to see the installation logs

# Install the dependencies
!pip install -qU python-dotenv
!pip install -qU langchain-core==0.1.23
!pip install -qU langchain==0.1.6
!pip install -qU langchain-community==0.0.19
!pip install -qU langchain-openai
!pip install --upgrade --quiet  transformers --quiet
!pip install -qU newspaper3k


# langchain==0.1.6
# langchain-community==0.0.19
# langchain-core==0.1.23

In [2]:
# Load environment variables
import os
from dotenv import load_dotenv, find_dotenv

load_dotenv(find_dotenv())

True

In [3]:
# Not a good practice, but we will ignore warnings in this notebook, as tensor has deprecated some methods and will be removed in future versions.
# https://github.com/pytorch/pytorch/issues/97207#issuecomment-1494781560
import warnings

# warnings.filterwarnings('ignore')
warnings.filterwarnings(
    "ignore", category=UserWarning, message="TypedStorage is deprecated"
)

Update your `API_KEY` in the `.env` file. You can get the API keys from the following links. _Note: Some of the services may require you to have an account and some may charge you for usage_
- [OpenAI API Key](https://platform.openai.com/account/api-keys)
- [Hugging Face API Key](https://huggingface.co/settings/tokens)
- [Serper API Key](https://serper.dev/api-key)

In [4]:
# os.environ["HF_TOKEN"] = ""
# os.environ["HUGGINGFACEHUB_API_TOKEN"] = ""
# os.environ["OPENAI_API_KEY"] = ""
# os.environ["SERPER_API_KEY"] = ""

In [5]:
from langchain_openai import OpenAI
from langchain_openai import ChatOpenAI

# To specify a particular model refer to the OpenAI documentation - https://platform.openai.com/docs/models
# Completions Model: https://platform.openai.com/docs/models/completions
# Chat Model: https://platform.openai.com/docs/models/completions

llm = OpenAI()
llm_chat = ChatOpenAI(model_name="gpt-3.5-turbo-0125", temperature=0.3)

In [6]:
from langchain_community.utilities import GoogleSerperAPIWrapper
from langchain.chains.summarize import load_summarize_chain
from langchain.docstore.document import Document
from langchain.prompts import PromptTemplate
from langchain.text_splitter import RecursiveCharacterTextSplitter
from langchain.document_loaders import NewsURLLoader

**Serper API** - [Sign up](https://serper.dev/signup?ref=miztiik) for an account with Serper, or log in if you already have an account, and create an API key. Serper offers a generous free tier; as you consume the API, the dashboard will populate with the requests and remaining credits.

In [7]:
import os

search = GoogleSerperAPIWrapper(
    type="news", tbs="qdr:d1", serper_api_key=os.getenv("SERPER_API_KEY")
)

news_search_query = "global ai growth"
news_results = search.results(news_search_query, num_results=5)

if news_results.get("news") is None:
    print("No results found")

In [8]:
print(len(news_results["news"]))

10


In [9]:
for i in news_results["news"][0]:
    print(f"{i}:{news_results['news'][0][i]}")

title:Missed Out on Nvidia? 1 Artificial Intelligence (AI) Growth Stock to Buy Now and Hold for a Decade or Longer
link:https://finance.yahoo.com/news/missed-nvidia-1-artificial-intelligence-112700264.html
snippet:Semiconductor stocks have been on fire ever since ChatGPT was launched in late 2022. Since then, a slew of new generative artificial intelligence (AI)...
date:3 hours ago
source:Yahoo Finance
imageUrl:https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0XN6yi_CzyI6foZHOdoCHmwiWUsQheXzMR4gZjDakvYnoN9UWCtaoYk4b1A&s
position:1


In [10]:
# Limit how many news articles to process
num_results = min(5, len(news_results["news"]))

text_splitter = RecursiveCharacterTextSplitter(
    separators=[
        "\n\n",
        "\n",
    ],
    chunk_size=1000,
    chunk_overlap=100,
)

# For each news article, load the contents
for index, news_item in enumerate(news_results["news"]):
    loader = NewsURLLoader(urls=[news_item.get("link")])
    contents = loader.load()
    if contents:
        news_results["news"][index]["article"] = contents
        # Make the docs to fit model input size
        news_results["news"][index]["split_article"] = text_splitter.create_documents(
            [contents[0].page_content]
        )
    else:
        print(f"Failed to load {news_item['link']}, removed from results.\n")
        news_results["news"].pop(index)

Error fetching or processing https://aithority.com/it-and-devops/data-management/66degrees-achieves-6x-ai-revenue-growth-expands-data-and-ai-transformation-offerings-globally/, exception: Article `download()` failed with HTTPSConnectionPool(host='aithority.com', port=443): Read timed out. (read timeout=7) on URL https://aithority.com/it-and-devops/data-management/66degrees-achieves-6x-ai-revenue-growth-expands-data-and-ai-transformation-offerings-globally/


Failed to load https://aithority.com/it-and-devops/data-management/66degrees-achieves-6x-ai-revenue-growth-expands-data-and-ai-transformation-offerings-globally/, removed from results.



In [11]:
for i in news_results["news"]:
    print(i)
    print(f"\033[32m-----\033[0m")

{'title': 'Missed Out on Nvidia? 1 Artificial Intelligence (AI) Growth Stock to Buy Now and Hold for a Decade or Longer', 'link': 'https://finance.yahoo.com/news/missed-nvidia-1-artificial-intelligence-112700264.html', 'snippet': 'Semiconductor stocks have been on fire ever since ChatGPT was launched in late 2022. Since then, a slew of new generative artificial intelligence (AI)...', 'date': '3 hours ago', 'source': 'Yahoo Finance', 'imageUrl': 'https://encrypted-tbn0.gstatic.com/images?q=tbn:ANd9GcT0XN6yi_CzyI6foZHOdoCHmwiWUsQheXzMR4gZjDakvYnoN9UWCtaoYk4b1A&s', 'position': 1, 'article': [Document(page_content='Semiconductor stocks have been on fire ever since ChatGPT was launched in late 2022. Since then, a slew of new generative artificial intelligence (AI) applications have made cutting-edge graphics processing units that can handle accelerated applications a hot commodity. As Nvidia (NASDAQ: NVDA) is the leader in that subset of the chip market, its sales and stock price have been 

## Summarization with Open AI Models

<img src="images/miztiik_text_summarization_02.png" width="50%"/>

If our document is short enough to fit within the context window of the model, we can use the `stuffing` method.

**Stuffing method** - The `stuffing` method is the easiest way to summarize text by feeding the entire document to a large language model (LLM) in a single call. This method has both pros and cons.

  - **Pros**:
    - Only required a single call to the model, which can be faster than other methods that require multiple calls
    - When summarizing text, the model has access to all the data at once, which can result in a better summary.
  - **Cons**:
    - Most models have a context length, and for large documents (or many documents) this will not work as it will result in a prompt larger than the context length.
    - This method only works on smaller pieces of data and not suitable to large documents most of the time.

In [13]:
# 16k is the max input length for GPT-3.5
# num_tokens_first_doc = llm.get_num_tokens(
#     news_results["news"][1]["contents"][0].page_content
# )

map_prompt = """ Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(
    template=map_prompt,
    input_variables=["text"]
)

combine_prompt = """ Write a concise summary of the following text delimited by triple backquotes.
```{text}```
CONCISE SUMMARY:
"""
combine_prompt_template = PromptTemplate(
    template=combine_prompt,
    input_variables=["text"]
)

oai_chain = load_summarize_chain(
    llm=llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    # Uncomment verbose=True if you want to see the prompts being used
    # verbose=True
)

for news_item in news_results["news"][:num_results]:
    if news_item.get("article"):
        print(
            f"Summarizing article: {news_item['title']} - {news_item['link']}\n")
        news_item["oai_summary"] = oai_chain.invoke(news_item["split_article"])

Let us take a look at the summaries generated by the `map-reduce` method.

In [16]:
for i in news_results["news"][:num_results]:
    if i.get("article"):
        print(
            f"\nTitle: {i['title']}\nLink: {i['link']}\nSummary: \033[32m{i['oai_summary']['output_text']}\033[0m"
        )


Title: Missed Out on Nvidia? 1 Artificial Intelligence (AI) Growth Stock to Buy Now and Hold for a Decade or Longer
Link: https://finance.yahoo.com/news/missed-nvidia-1-artificial-intelligence-112700264.html
Summary: 
The launch of ChatGPT in 2022 has boosted the semiconductor industry, particularly in the GPU market, leading to increased sales and stock prices for Nvidia. However, some investors are concerned about the company's high valuation and the cyclical nature of the industry. An alternative investment in Alphabet, with its strong AI capabilities, is suggested. Alphabet's reliable earnings from advertising and cloud services make it a smart long-term investment, and the company is currently undervalued. The Motley Fool recommends considering other investment options, but acknowledges the potential for high earnings from Alphabet, which is expected to continue strong growth in its two main segments for at least another decade. 

Title: Hon Hai sees AI servers as sales driver in

### Observations

The summarization is reads like written by a person, does a good job of capturing the main points of the article. The summary is coherent and reads well. As OpenAI continues to improve their models, we can expect the quality of the summaries to improve as well.

## Summarization with Huggingface Open Source Hosted Models with LangChain

<img src="images/miztiik_text_summarization_03.png" width="50%"/>

We will try a variety of model and see how they perform. We will use,
- `google/flan-t5-xxl`
- `facebook/bart-large-cnn`
- `sshleifer/distilbart-cnn-12-6`
- `Falconsai/text_summarization`

### Summarization with Hugging Face hosted models

In [17]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms.huggingface_hub import HuggingFaceHub

hf_hub_llm = HuggingFaceHub(
    repo_id="google/flan-t5-xxl",
    model_kwargs={"temperature": 0.3, "max_length": 512},
    # repo_id="philschmid/bart-large-cnn-samsum", model_kwargs={"temperature": 0.3, "max_length": 256}
    # repo_id="mistralai/Mistral-7B-v0.1", model_kwargs={"temperature": 0.3, "max_length": 512}
)

hf_hub_chain = load_summarize_chain(
    llm=hf_hub_llm,
    chain_type="map_reduce",
    # verbose=True
)

# run chain
for news_item in news_results["news"][:num_results]:
    if news_item.get("article"):
        print(
            f"Summarizing article: {news_item['title']} - {news_item['link']}\n")
        news_item["hf_hub_summary"] = hf_hub_chain.invoke(
            news_item["split_article"])

Summarizing article: Missed Out on Nvidia? 1 Artificial Intelligence (AI) Growth Stock to Buy Now and Hold for a Decade or Longer - https://finance.yahoo.com/news/missed-nvidia-1-artificial-intelligence-112700264.html

Summarizing article: Hon Hai sees AI servers as sales driver in 2024 - https://focustaiwan.tw/business/202402170013

Summarizing article: A Look Into the Global AI Regulatory Landscape - https://www.project-disco.org/innovation/a-look-into-the-global-ai-regulatory-landscape/

Summarizing article: OpenAI CEO Sam Altman's Bold Venture to Boost Global AI Chip Manufacturing - https://medriva.com/business/openai-ceo-sam-altmans-bold-venture-to-boost-global-ai-chip-manufacturing/



In [18]:
for i in news_results["news"][:num_results]:
    if i.get("article"):
        print(
            f"\nTitle: {i['title']}\nLink: {i['link']}\nSummary: \033[32m{i['hf_hub_summary']['output_text']}\033[0m"
        )


Title: Missed Out on Nvidia? 1 Artificial Intelligence (AI) Growth Stock to Buy Now and Hold for a Decade or Longer
Link: https://finance.yahoo.com/news/missed-nvidia-1-artificial-intelligence-112700264.html
Summary: Nvidia's stock has been on fire, but investors are rightly nervous that the stock has gotten too far ahead of itself. Nvidia is a great company, but its stock is overvalued. Alphabet is the world's largest AI company. Google is a leader in AI, and it's a leader in the development of AI talent. Google Cloud is a fast-growing business that's expected to grow even faster in the future

Title: Hon Hai sees AI servers as sales driver in 2024
Link: https://focustaiwan.tw/business/202402170013
Summary: Hon Hai Precision Industry Co., the world's largest contract manufacturer of electronic components, has announced that it will invest NT$10 billion (US$33 million) in the development of artificial intelligence (AI) applications and electric vehicles (EVs) in the next three years.


### Summarization with Huggingface Open Source Local Models with LangChain

In [19]:
from transformers import BartForConditionalGeneration, BartTokenizer
from transformers import AutoModelForSeq2SeqLM, AutoTokenizer
from transformers import pipeline

# Load the model and tokenizer

bart_model = AutoModelForSeq2SeqLM.from_pretrained("facebook/bart-large-cnn")
bart_tokenizer = AutoTokenizer.from_pretrained("facebook/bart-large-cnn")

print(f"bart_model_max_model_length:{bart_tokenizer.model_max_length}")

bart_model_max_model_length:1024


In [46]:
from langchain_community.llms.huggingface_pipeline import HuggingFacePipeline

bart_summarizer = pipeline(
    "summarization",
    model="facebook/bart-large-cnn",
    tokenizer="facebook/bart-large-cnn",
)

hf_bart_llm = HuggingFacePipeline(pipeline=bart_summarizer, model_kwargs={})


map_prompt = """ Write a concise summary of the following:
"{text}"
CONCISE SUMMARY:
"""
map_prompt_template = PromptTemplate(
    template=map_prompt,
    input_variables=["text"]
)

combine_prompt = """ Write a concise summary of the following text delimited by triple backquotes.
```{text}```
CONCISE SUMMARY:
"""
combine_prompt_template = PromptTemplate(
    template=combine_prompt,
    input_variables=["text"]
)


hf_bart_chain = load_summarize_chain(
    llm=hf_bart_llm,
    chain_type="map_reduce",
    map_prompt=map_prompt_template,
    combine_prompt=combine_prompt_template,
    verbose=True
)

# run chain
# for news_item in news_results["news"][:num_results]:
for news_item in news_results["news"][2:num_results]:
    if news_item.get("article"):
        print(
            f"Summarizing article: {news_item['title']} - {news_item['link']}\n")
        news_item["hf_bart_summary"] = hf_bart_chain.invoke(
            news_item["split_article"])
        print("!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!!")

Summarizing article: A Look Into the Global AI Regulatory Landscape - https://www.project-disco.org/innovation/a-look-into-the-global-ai-regulatory-landscape/



> Entering new MapReduceDocumentsChain chain...


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise summary of the following:
"A Look Into the Global AI Regulatory Landscape

Artificial intelligence (AI) is a highly versatile technology. From one program’s application in a pastry shop to its later employment in detecting cancer cells, AI is truly integrating itself into many diverse aspects of our world. However, the rapid rise of AI has also intensified the need for governments to strengthen their capabilities to understand, leverage, operate, and when it calls for it, regulate this technology. As a result, there is a growing regulatory tension between different jurisdictions, which includes the development of comprehensive or sector-specific legislation and voluntary guidelines and standards."
CONCIS

Your max_length is set to 142, but your input_length is only 125. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=62)
Your max_length is set to 142, but your input_length is only 73. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=36)
Your max_length is set to 142, but your input_length is only 132. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Your max_length is set to 142, but your input_length is only 133. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=66)
Y


> Finished chain.


> Entering new LLMChain chain...
Prompt after formatting:
 Write a concise summary of the following text delimited by triple backquotes.
```Artificial intelligence (AI) is a highly versatile technology. There is a growing regulatory tension between different jurisdictions. Governments need to strengthen their capabilities to understand, leverage, operate, and when it calls for it, regulate this technology. Write a concise summary of the following:"A Look Into the Global AI Regulatory Landscape"

The Biden-Harris Administration released its long anticipated Executive Order (EO) on AI. In Europe, EU member states recently approved final wording of the AI Act. While the EU sees the potential of AI through its own competitive AI ecosystem, many of the new AI rules remain unclear.

"The White House EO on AI does raise some concerns, but it also represents a positive step forward in AI governance and promises a bright future" "The Biden-Harris Administration is acutely a

IndexError: index out of range in self

In [47]:
news_results["news"][2]["split_article"]

[Document(page_content='A Look Into the Global AI Regulatory Landscape\n\nArtificial intelligence (AI) is a highly versatile technology. From one program’s application in a pastry shop to its later employment in detecting cancer cells, AI is truly integrating itself into many diverse aspects of our world. However, the rapid rise of AI has also intensified the need for governments to strengthen their capabilities to understand, leverage, operate, and when it calls for it, regulate this technology. As a result, there is a growing regulatory tension between different jurisdictions, which includes the development of comprehensive or sector-specific legislation and voluntary guidelines and standards.'),
 Document(page_content='In June of last year, Josh Landau discussed AI legislation in the U.S. and the effects it could have on the industry. And in late October, the Biden-Harris Administration finally released its long anticipated Executive Order (EO) on AI. In the wake of the NIST Risk Ma

In [ ]:
# news_results["news"][2]["split_article"]

for i in news_results["news"][2]["split_article"]:
    print(i)
    print(hf_bart_chain.invoke([i]))

In [161]:
# Print the summaries
for i in news_results["news"][:num_results]:
    if i.get("article"):
        print(f"\nTitle: {i['title']}\nLink: {i['link']}")
        print(
            f"\noai_summary: \033[32m {i['oai_summary']['output_text']}\033[0m")
        print(
            f"\nhf_hub_summary: \033[32m{i['hf_hub_summary']['output_text']}\033[0m")
        print(
            f"\nhf_bart_summary: \033[32m{i['hf_bart_summary']['output_text']}\033[0m"
        )


Title: The global economy in 2024: World Bank predicts slowest half-decade of growth in 30 years - People & Profit
Link: https://www.france24.com/en/tv-shows/people-profit/20240216-the-global-economy-in-2024-world-bank-predicts-slowest-half-decade-of-growth-in-30-years

oai_summary:   The World Bank predicts slow economic growth for the next five years, which could hinder development and climate goals. Developing and low-income economies will be most affected. The World Bank's Deputy Chief Economist talks about these findings in an interview with FRANCE 24.

hf_hub_summary: The World Bank has warned that the global economy is set to grow at a rate of just 2% per year for the next five years, a far cry from the 4% growth it predicted in its Global Economic Prospects report.

hf_bart_summary: World Bank predicts slowest half-decade of growth in 30 years. This is having devastating consequences, especially in developing and low-income economies. For more, FRANCE 24's Charles Pellegrin sp

### Summarization with Huggingface Open Source Smaller Local Models with LangChain


In [163]:
hf_distilbart_summarizer = pipeline(
    "summarization", model="sshleifer/distilbart-cnn-12-6"
)

hf_distilbart_llm = HuggingFacePipeline(
    pipeline=hf_distilbart_summarizer, model_kwargs={}
)


hf_distilbart_chain = load_summarize_chain(
    llm=hf_distilbart_llm,
    chain_type="map_reduce",
    # verbose=True
)

# run chain
for news_item in news_results["news"][:num_results]:
    if news_item.get("article"):
        print(
            f"Summarizing article: {news_item['title']} - {news_item['link']}\n")
        news_item["hf_distilbart_summary"] = hf_distilbart_chain.invoke(
            news_item["split_article"]
        )

Summarizing article: The global economy in 2024: World Bank predicts slowest half-decade of growth in 30 years - People & Profit - https://www.france24.com/en/tv-shows/people-profit/20240216-the-global-economy-in-2024-world-bank-predicts-slowest-half-decade-of-growth-in-30-years



Your max_length is set to 142, but your input_length is only 77. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=38)


Summarizing article: The Coming Drag on Economic Growth From Security Competition in Asia and Europe - https://worldview.stratfor.com/article/coming-drag-economic-growth-security-competition-asia-and-europe



Your max_length is set to 142, but your input_length is only 97. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=48)


Summarizing article: Russia's Invasion of Ukraine Threatens Indonesia's Economy - https://www.businessinsider.com/sc/russias-invasion-of-ukraine-threatens-indonesias-economy

Summarizing article: Recession has struck some of the world's top economies. The US keeps defying expectations - https://abcnews.go.com/US/wireStory/recession-struck-worlds-top-economies-us-defying-expectations-107272969

Summarizing article: Japan slips into a recession and loses its spot as the world's third-largest economy - https://apnews.com/article/japan-economy-2023-gdp-893d53deba654c4924e4924f0b321cc5



Your max_length is set to 142, but your input_length is only 121. Since this is a summarization task, where outputs shorter than the input are typically wanted, you might consider decreasing max_length manually, e.g. summarizer('...', max_length=60)


In [165]:
# Print the summaries
for i in news_results["news"][:num_results]:
    if i.get("article"):
        print(f"\nTitle: {i['title']}\nLink: {i['link']}")
        print(
            f"oai_summary: \033[32m {i['oai_summary']['output_text']}\033[0m")
        print(
            f"hf_hub_summary: \033[32m{i['hf_hub_summary']['output_text']}\033[0m")
        print(
            f"hf_bart_summary: \033[32m{i['hf_bart_summary']['output_text']}\033[0m")
        print(
            f"hf_distilbart_summary: \033[32m{i['hf_distilbart_summary']['output_text']}\033[0m"
        )


Title: The global economy in 2024: World Bank predicts slowest half-decade of growth in 30 years - People & Profit
Link: https://www.france24.com/en/tv-shows/people-profit/20240216-the-global-economy-in-2024-world-bank-predicts-slowest-half-decade-of-growth-in-30-years
oai_summary:   The World Bank predicts slow economic growth for the next five years, which could hinder development and climate goals. Developing and low-income economies will be most affected. The World Bank's Deputy Chief Economist talks about these findings in an interview with FRANCE 24.
hf_hub_summary: The World Bank has warned that the global economy is set to grow at a rate of just 2% per year for the next five years, a far cry from the 4% growth it predicted in its Global Economic Prospects report.
hf_bart_summary: World Bank predicts slowest half-decade of growth in 30 years. This is having devastating consequences, especially in developing and low-income economies. For more, FRANCE 24's Charles Pellegrin speak

### Summarization with Huggingface Open Source Smaller Local Models with LangChain - Falconsai

In [ ]:
hf_falconsai_summarizer = pipeline(
    "summarization", model="Falconsai/text_summarization"
)

hf_falconsai_llm = HuggingFacePipeline(
    pipeline=hf_falconsai_summarizer, model_kwargs={}
)


hf_falconsai_chain = load_summarize_chain(
    llm=hf_falconsai_llm,
    chain_type="map_reduce",
    # verbose=True
)

# run chain
for news_item in news_results["news"][:num_results]:
    if news_item.get("article"):
        print(
            f"Summarizing article: {news_item['title']} - {news_item['link']}\n")
        news_item["hf_falconsai_summary"] = hf_falconsai_chain.invoke(
            news_item["split_article"]
        )

In [167]:
# Print the summaries
for i in news_results["news"][:num_results]:
    if i.get("article"):
        print(f"\nTitle: {i['title']}\nLink: {i['link']}")
        print(
            f"oai_summary: \033[32m {i['oai_summary']['output_text']}\033[0m")
        print(
            f"hf_hub_summary: \033[32m{i['hf_hub_summary']['output_text']}\033[0m")
        print(
            f"hf_bart_summary: \033[32m{i['hf_bart_summary']['output_text']}\033[0m")
        print(
            f"hf_distilbart_summary: \033[32m{i['hf_distilbart_summary']['output_text']}\033[0m"
        )
        print(
            f"hf_falconsai_summary: \033[32m{i['hf_falconsai_summary']['output_text']}\033[0m"
        )


Title: The global economy in 2024: World Bank predicts slowest half-decade of growth in 30 years - People & Profit
Link: https://www.france24.com/en/tv-shows/people-profit/20240216-the-global-economy-in-2024-world-bank-predicts-slowest-half-decade-of-growth-in-30-years
oai_summary:   The World Bank predicts slow economic growth for the next five years, which could hinder development and climate goals. Developing and low-income economies will be most affected. The World Bank's Deputy Chief Economist talks about these findings in an interview with FRANCE 24.
hf_hub_summary: The World Bank has warned that the global economy is set to grow at a rate of just 2% per year for the next five years, a far cry from the 4% growth it predicted in its Global Economic Prospects report.
hf_bart_summary: World Bank predicts slowest half-decade of growth in 30 years. This is having devastating consequences, especially in developing and low-income economies. For more, FRANCE 24's Charles Pellegrin speak

### Summarization with Huggingface Open Source Smaller Local Models with LangChain - MistralAI Hosted

In [1]:
from langchain.chains.summarize import load_summarize_chain
from langchain.llms.huggingface_hub import HuggingFaceHub

hf_hub_llm = HuggingFaceHub(
    repo_id="mistralai/Mixtral-8x7B-Instruct-v0.1",
    model_kwargs={"temperature": 0.3, "max_length": 512},
)

hf_hub_chain = load_summarize_chain(
    llm=hf_hub_llm,
    chain_type="map_reduce",
    # verbose=True
)

# run chain
# for news_item in news_results["news"][:num_results]:
for news_item in news_results["news"][1:num_results]:
    if news_item.get("article"):
        print(
            f"Summarizing article: {news_item['title']} - {news_item['link']}\n")
        news_item["hf_hub_summary"] = hf_hub_chain.invoke(
            news_item["split_article"])

ValidationError: 1 validation error for HuggingFaceHub
__root__
  Did not find huggingfacehub_api_token, please add an environment variable `HUGGINGFACEHUB_API_TOKEN` which contains it, or pass `huggingfacehub_api_token` as a named parameter. (type=value_error)

In [ ]:
from transformers import AutoModelForCausalLM, AutoTokenizer, pipeline
import torch
from transformers import BitsAndBytesConfig
from langchain import HuggingFacePipeline
from langchain import PromptTemplate, LLMChain


quantization_config = BitsAndBytesConfig(
    load_in_4bit=True,
    bnb_4bit_compute_dtype=torch.float16,
    bnb_4bit_quant_type="nf4",
    bnb_4bit_use_double_quant=True,
)
model_id = "mistralai/Mistral-7B-Instruct-v0.1"

model_4bit = AutoModelForCausalLM.from_pretrained(
    model_id,
    device_map="auto",
    quantization_config=quantization_config,
)
tokenizer = AutoTokenizer.from_pretrained(model_id)

pipeline = pipeline(
    "text-generation",
    model=model_4bit,
    tokenizer=tokenizer,
    use_cache=True,
    device_map="auto",
    max_length=500,
    do_sample=True,
    top_k=5,
    num_return_sequences=1,
    eos_token_id=tokenizer.eos_token_id,
    pad_token_id=tokenizer.eos_token_id,
)

In [ ]:
# Print the summaries
for i in news_results["news"][:num_results]:
    if i.get("article"):
        print(f"\nTitle: {i['title']}\nLink: {i['link']}")
        print(
            f"oai_summary: \033[32m {i['oai_summary']['output_text']}\033[0m")
        print(
            f"hf_hub_summary: \033[32m{i['hf_hub_summary']['output_text']}\033[0m")
        print(
            f"hf_bart_summary: \033[32m{i['hf_bart_summary']['output_text']}\033[0m")
        print(
            f"hf_distilbart_summary: \033[32m{i['hf_distilbart_summary']['output_text']}\033[0m"
        )
        print(
            f"hf_falconsai_summary: \033[32m{i['hf_falconsai_summary']['output_text']}\033[0m"
        )

## Additional Reading

1. [LLM Bootcamp](https://github.com/miztiik/llm-bootcamp)
1. [Revolutionizing News Summarization](https://www.width.ai/post/revolutionizing-news-summarization-exploring-the-power-of-gpt-in-zero-shot-and-specialized-tasks)
1. [Summarizer For Any Size Document](https://www.width.ai/post/gpt3-summarizer)
1. [Langchain Summarization 1. Stuff & Map Reduce](https://python.langchain.com/docs/use_cases/summarization)
1. [Langchain Google Serper](https://python.langchain.com/docs/integrations/tools/google_serper)
1. [Hugging Face Local Pipelines](https://python.langchain.com/docs/integrations/llms/huggingface_pipelines)
1. [Chunking Strategies for LLM Applications](https://www.pinecone.io/learn/chunking-strategies/)
1. [Optimal Chunk-Size for Large Document Summarization](https://vectify.ai/blog/LargeDocumentSummarization)
1 .[4 Powerful Long Text Summarization Methods With Real Examples](https://www.width.ai/post/4-long-text-summarization-methods)

In [ ]:
prompt_template = """Write a concise summary of the following:


{text}


CONCISE SUMMARY:"""
PROMPT = PromptTemplate(template=prompt_template, input_variables=["text"])
refine_template = (
    "Your job is to produce a final summary\n"
    "We have provided an existing summary up to a certain point: {existing_answer}\n"
    "We have the opportunity to refine the existing summary"
    "(only if needed) with some more context below.\n"
    "------------\n"
    "{text}\n"
    "------------\n"
    "Given the new context, refine the original summary. Make sure to address the list of problems, list of solutions and any following action"
    "If the context isn't useful, return the original summary."
)
refine_prompt = PromptTemplate(
    input_variables=["existing_answer", "text"],
    template=refine_template,
)
chain = load_summarize_chain(
    llm,
    chain_type="refine",
    return_intermediate_steps=True,
    question_prompt=PROMPT,
    refine_prompt=refine_prompt,
)
chain({"input_documents": docs}, return_only_outputs=True)